In [1]:
!pip install plotly

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans, DBSCAN
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe_connected"
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score

# Dataset: Uber rides in April 2014 - where one observation is one Uber ride starting point
df = pd.read_csv('uber-raw-data-apr14.csv')

# Create a column with latitude and longitude joined
df['coords'] = [', '.join(str(x) for x in y) for y in map(tuple, df[['Lat', 'Lon']].values)]

# Getting the day, month, hour and minute of every observation from the date/time column
df['Date/Time']= pd.to_datetime(df['Date/Time'])

df['Day'], df['Weekday'], df['Hour'], df['Minute'] = df['Date/Time'].dt.day, df['Date/Time'].dt.dayofweek, df['Date/Time'].dt.hour, df['Date/Time'].dt.minute

# Renaming the weekdays
df['Weekday'] = df['Weekday'].apply(lambda x: 
                                "Monday" if x == 0
                                else "Tuesday" if x == 1
                                else "Wednesday" if x == 2
                                else "Thursday" if x == 3
                                else "Friday" if x == 4
                                else "Saturday" if x == 5
                                else "Sunday")

# Dropping the Date/Time and Base columns
df = df.drop(['Date/Time', 'Base'], axis=1)

# Changing the type of the Hour column to integer
df['Hour'] = df['Hour'].astype('int')

# Hot zones = Getting the 1000 coordinates that have the most occurences = most calls in the month
n = 1000
list_z = df['coords'].value_counts()[:n].index.tolist()

# Creating an empty column 
df['hot_z'] = ''

# Writing 'Yes' at every row if the coordinates are in the top 1000
for i in range(len(df)):
    for j in range(len(list_z)):
        if df['coords'][i] == list_z[j]:
            df['hot_z'][i] = 'Yes'

# Creating a new df with hot zones only
hot_zones = df[df['hot_z'] == 'Yes']

# Saving it into a csv
hot_zones.to_csv('hot_zones_apr14.csv')

In [3]:
hot_zones = pd.read_csv('hot_zones_apr14.csv')
hot_zones

,Unnamed: 0,Lat,Lon,coords,Day,Weekday,Hour,Minute,hot_z
0,18,40.7531,-74.0039,"40.7531, -74.0039",1,Tuesday,3,35,Yes
1,24,40.6950,-74.1783,"40.695, -74.1783",1,Tuesday,4,20,Yes
2,26,40.6879,-74.1814,"40.6879, -74.1814",1,Tuesday,4,27,Yes
3,27,40.6878,-74.1816,"40.6878, -74.1816",1,Tuesday,4,38,Yes
4,39,40.7217,-73.9875,"40.7217, -73.9875",1,Tuesday,5,36,Yes
...,...,...,...,...,...,...,...,...,...
48386,564462,40.6449,-73.7821,"40.6449, -73.7821",30,Wednesday,21,30,Yes
48387,564484,40.7209,-74.0042,"40.7209, -74.0042",30,Wednesday,22,14,Yes
48388,564497,40.7325,-73.9969,"40.7325, -73.9969",30,Wednesday,22,42,Yes
48389,564509,40.7420,-74.0037,"40.742, -74.0037",30,Wednesday,23,15,Yes


In [4]:
# Dropping unnecessary columns
hot_zones = hot_zones.drop(columns=['Unnamed: 0'], axis=1)

In [5]:
hot_zones

,Lat,Lon,coords,Day,Weekday,Hour,Minute,hot_z
0,40.7531,-74.0039,"40.7531, -74.0039",1,Tuesday,3,35,Yes
1,40.6950,-74.1783,"40.695, -74.1783",1,Tuesday,4,20,Yes
2,40.6879,-74.1814,"40.6879, -74.1814",1,Tuesday,4,27,Yes
3,40.6878,-74.1816,"40.6878, -74.1816",1,Tuesday,4,38,Yes
4,40.7217,-73.9875,"40.7217, -73.9875",1,Tuesday,5,36,Yes
...,...,...,...,...,...,...,...,...
48386,40.6449,-73.7821,"40.6449, -73.7821",30,Wednesday,21,30,Yes
48387,40.7209,-74.0042,"40.7209, -74.0042",30,Wednesday,22,14,Yes
48388,40.7325,-73.9969,"40.7325, -73.9969",30,Wednesday,22,42,Yes
48389,40.7420,-74.0037,"40.742, -74.0037",30,Wednesday,23,15,Yes


In [6]:
# Data preprocessing 

In [7]:
# We want our model to identify clusters on coordinates. To do so, we isolate our Latitudes and Longitudes in an object X.
X = hot_zones.loc[:,['Lat', 'Lon']]

In [8]:
# Numeric features we need to standardize 
numeric_features = [0,1]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) 
])


preprocessor = ColumnTransformer(
   transformers=[
        ('num', numeric_transformer, numeric_features)
       
    ])

# Applying the preprocessing on the dataset
X = preprocessor.fit_transform(X) 

In [9]:
# We choose to use the elbow method to find the optimal number of clusters
wcss =  []
for i in range(2,10): 
    kmeans = KMeans(n_clusters= i)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
print(wcss)

[31683.220555346994, 13889.079364576308, 7326.209249366298, 2777.715958576979, 1466.9127018636975, 1158.7714493273827, 898.0870435355812, 786.3739312655513]


In [10]:
# Plotting the results 
fig = px.line(x = range(2,10), y = wcss)
fig.show()

In [11]:
# By looking at the result of the elbow method, the optimal number of clusters seems to be around 4.
kmeans = KMeans(n_clusters= 4, random_state=0, init="k-means++")

# Time to fit the model on our normalized data
kmeans.fit(X)

KMeans(n_clusters=4, random_state=0)

In [12]:
# Then we add our newly created clusters to our original dataframe:
label = kmeans.predict(X)
hot_zones['cluster'] = label
hot_zones.head()

,Lat,Lon,coords,Day,Weekday,Hour,Minute,hot_z,cluster
0,40.7531,-74.0039,"40.7531, -74.0039",1,Tuesday,3,35,Yes,2
1,40.6950,-74.1783,"40.695, -74.1783",1,Tuesday,4,20,Yes,3
2,40.6879,-74.1814,"40.6879, -74.1814",1,Tuesday,4,27,Yes,3
3,40.6878,-74.1816,"40.6878, -74.1816",1,Tuesday,4,38,Yes,3
4,40.7217,-73.9875,"40.7217, -73.9875",1,Tuesday,5,36,Yes,2


In [13]:
# How many observations do we have per cluster?
hot_zones['cluster'].value_counts()

2    33702
0     7156
1     6424
3     1109
Name: cluster, dtype: int64

In [14]:
# Visualization of our clusters by weekday.
fig = px.scatter_mapbox(hot_zones, lat="Lat", lon="Lon",  color='cluster', size='Hour' , zoom = 9,
                        mapbox_style="open-street-map", color_continuous_scale = 'Plasma',
                       animation_frame = 'Weekday', title='Hot Zones by Weekday and Hour')
                    
fig.show()

In [15]:
# We can see that there is not a significant difference in the number of calls day by day.

In [16]:
# Now we look at the density (= number of uber calls) for every hour
fig = px.density_mapbox(hot_zones, lat="Lat", lon="Lon", mapbox_style="open-street-map", 
                        animation_frame = 'Hour',
                        zoom = 10, radius = 8, hover_name= 'cluster')
fig.show()

In [17]:
# There is more to discover when we look at the calls on a day, hour by hour! 

In [18]:
# Finding the number of calls per hour and per cluster
test2 = hot_zones.groupby(['Hour','cluster']).cluster.agg('count').to_frame('count').reset_index()

In [19]:
test2

,Hour,cluster,count
0,0,0,17
1,0,1,133
2,0,2,872
3,0,3,30
4,1,0,3
...,...,...,...
88,22,3,61
89,23,0,148
90,23,1,352
91,23,2,1462


In [20]:
import plotly.express as px

fig = px.bar(test2, x='Hour', y='count', color='cluster', title = 'Rides per hour by cluster on average in a day')
fig.show()

In [21]:
# Now we can clearly see that the drivers should definitely be more present in the cluster nb 2, especially between 2pm and 11pm!

In [22]:
# Kmeans seems to have performed quite well. What if we tried a different clustering model, like DBscan?

In [23]:
# Instanciate DBSCAN - several tries before finding these parameters!
db = DBSCAN(eps=0.3, min_samples=50, metric="manhattan", algorithm="auto")

In [24]:
# Fit on data
db.fit(X)

DBSCAN(eps=0.3, metric='manhattan', min_samples=50)

In [25]:
# Visualization of our clusters with DBScan 
import plotly.graph_objects as go

fig = go.Figure()

for i in np.unique(db.labels_):
    label = X[db.labels_ == i]
    fig.add_trace(go.Scatter(x=label[:, 0], y=label[:, 1], mode="markers", name="Cluster {}".format(i)))

fig.show()

In [26]:
# Adding the DBScan clusters to the dataset
label2 = db.labels_
hot_zones['cluster2'] = label2
hot_zones.head()

,Lat,Lon,coords,Day,Weekday,Hour,Minute,hot_z,cluster,cluster2
0,40.7531,-74.0039,"40.7531, -74.0039",1,Tuesday,3,35,Yes,2,0
1,40.6950,-74.1783,"40.695, -74.1783",1,Tuesday,4,20,Yes,3,1
2,40.6879,-74.1814,"40.6879, -74.1814",1,Tuesday,4,27,Yes,3,1
3,40.6878,-74.1816,"40.6878, -74.1816",1,Tuesday,4,38,Yes,3,1
4,40.7217,-73.9875,"40.7217, -73.9875",1,Tuesday,5,36,Yes,2,0


In [27]:
# Visualization of the DBScan cluster on a map : 
# Like with Kmeans, there's one major cluster and several smaller ones.
fig = px.scatter_mapbox(hot_zones, lat="Lat", lon="Lon", size="Hour", color='cluster2', zoom = 9,
                        mapbox_style="open-street-map", color_continuous_scale = 'Plasma',
                       animation_frame = 'Weekday', title='Hot Zones by Hour and Weekday')
fig.show()

In [28]:
# Finding the number of calls per hour and cluster with DBscan
test = hot_zones.groupby(['Hour','cluster2']).cluster2.agg('count').to_frame('count').reset_index()

In [29]:
fig = px.bar(test, x='Hour', y='count', color='cluster2', title = 'Rides per hour by cluster on average in a day with DBScan')
fig.show()

In [30]:
# Results with DBScan are similar to Kmeans, but a bit too precise : 
# some cluster don't have enough observations to appear on the graph compared to other clusters.
# We decide to stick with Kmeans for this business case.

In [31]:
!pip install geopy

In [32]:
from geopy.geocoders import Nominatim
import time
import re
geolocator = Nominatim(timeout=10, user_agent='http')

In [35]:
hot_zones['Zone_Name'] = ''

In [52]:
for i in range(len(hot_zones)):
    location = geolocator.reverse(hot_zones['coords'][i])
    address = location.raw['address']
    suburb = address.get('suburb')
    city = address.get('city')
    hot_zones['Zone_Name'][i] = suburb, city
    time.sleep(1)

<ipython-input-52-d0fd5d0f023f>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyboardInterrupt: 

In [ ]:
location = geolocator.reverse('40.6879, -74.1814')
address = location.raw['address']
suburb = address.get('suburb')
address

In [64]:
hot_zones['Zone_Name'].value_counts()

                         30310
(Manhattan, New York)    10492
(Queens, New York)        4788
(Manhattan, None)         2114
(None, Newark)             424
(Brooklyn, New York)       117
(Brooklyn, None)           117
(None, None)                29
Name: Zone_Name, dtype: int64

In [ ]:
# Hot Zones are d